# New Jersey School Performance Data and Taxpayers Guide to Education Spending Analysis

### Purpose
New Jersey provides multiple sets of public data on school systems to enable comparisons and informed decisions regarding school performance and spending. This notebook leverages two primary resources:
- **Taxpayers Guide to Education Spending**: [https://www.nj.gov/education/guide/](https://www.nj.gov/education/guide/)
- **New Jersey School Performance Report**: [https://rc.doe.state.nj.us/](https://rc.doe.state.nj.us/)

Various "Ranking" systems use different metrics to rank schools within a particular area. This compares similar districts by DFG and Group across the state:
- **Cost per pupil**
- **Test Results in ELA/Math**
- **Chronic Absentee Rates**

This analysis aims to compare "similar" schools to facilitate fairer comparisons. We use two main criteria to classify and compare districts:

1. **District Factor Groups (DFG)**: Demographic classification system for New Jersey school districts:
   - A (Abbott Districts)
   - B
   - CD
   - DE
   - FG
   - GH
   - I
   - J (Very affluent districts)

2. **School Size/Type**: Based on grade level and student population
   - A. K-6
   - B. K-8 / 0 - 400 students
   - C. K-8 / 401 - 750 students
   - D. K-8 / 751+ students
   - E. K-12 / 0 - 1800 students
   - F. K-12 / 1801 - 3500 students
   - G. K-12 / 3501+ students
   - H. 7-12 / 9-12
   - I. CSSD
   - J. VOC

For more on District Factor Groups, visit [NJ DFG](https://www.nj.gov/education/finance/rda/dfg.shtml).

### Analysis Output
This notebook provides multiple comparative views, including:
1. Schools in the **same county and group**
2. Schools in the **same county, group, and DFG**
3. Schools in the **same county**
4. Schools in the **same group and DFG + 1** (next higher district factor group)
5. Schools in **any other DFGs within the same group**
6. Schools in **any other counties within the same group**

For each comparison, the following metrics are displayed:
1. **ELA Scores** (by year)
2. **Math Scores** (by year)
3. **ELA Scores by year by graduation year (cohort)**
4. **Math Scores by year by graduation year (cohort)**
5. **Pupils per Administrator**
6. **Pupils per Staff** (Note: this is not classroom size)
7. **Per Pupil Cost**
8. **Staff Mean Salary**


## Set Report Parameters (This is what you change)

In [ ]:
# Set the district name you are comparing here:
dist_name = 'Hardyston Twp'
alt_dfg = [] # Ex. ['J'] # List of Alternate District Factor Groups to compare, we automatically add the next group up
alt_county = [] # Ex. ['Bergen'] # List of Alternate Counties to Compare
year = 2024 # Taxpayers Guide to Education Spending Year
# Drop down on the School Performance Report Page, Note missing 2 years due to covid
performance_years = [ '2022-2023', '2019-2020', '2018-2019', '2017-2018', '2016-2017'] 

## Setup Generic Functions

In [ ]:
import numpy as np
import pandas as pd
import fnmatch
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import requests
import zipfile
import io

def load_and_clean_data(filename, year_range):
    try:
        # Load all sheets into a dictionary
        sheet_names = pd.ExcelFile(filename).sheet_names
        # Define column selections and renaming based on sheet
        columns_mapping = {
            'ELAMathPerformanceTrends': ['SchoolYear', 'Subject', 'ProficiencyRateforFederalAccountability'],
            'MathPerformanceTrends': ['SchoolYear', 'GraduationYear', 'MetExcExpPerc', 'Subject'],
            'ELAPerformanceTrends': ['SchoolYear', 'GraduationYear', 'MetExcExpPerc', 'Subject'],
            'ChronicAbsByGrade': ['SchoolYear', 'Grade', 'DistrictPercent'],
            'ChronicAbsenteeismByGrade': ['SchoolYear', 'Grade', 'DistrictPercent']
        }
        column_variants = {
            'DIST': ['DistrictCode', 'DIST_CODE', 'DIST'],
            'Grades': ['Grades', 'Grades/Subject', 'Grade_Subject'],
            'MetExcExpPerc': ['% Met/Exceeded Expectation', 'MetExcExpPerc'],
            'ChronicAbsenteeismRate': ['ChronicAbsenteeismRate', 'Chronic Absenteeism Rate'],
            'DistrictPercent': ['DistrictPercent', 'SchoolPercent']
        }        
        # Initialize an empty dictionary to store cleaned data
        cleaned_data = {}
        for sheet_name in sheet_names:
            if sheet_name in columns_mapping:
                sheet_data = pd.read_excel(filename, sheet_name=sheet_name)
                for standard_col, possible_names in column_variants.items():
                    for name in possible_names:
                        if name in sheet_data.columns:
                            sheet_data.rename(columns={name: standard_col}, inplace=True)
                            break
                if 'SchoolYear' not in sheet_data.columns:
                    sheet_data["SchoolYear"] = year_range
                if "ProficiencyRateforFederalAccountability" in sheet_data.columns:
                    sheet_data["ProficiencyRateforFederalAccountability"] = pd.to_numeric(sheet_data["ProficiencyRateforFederalAccountability"], errors='coerce')
                if "MetExcExpPerc" in sheet_data.columns:
                    sheet_data["MetExcExpPerc"] = pd.to_numeric(sheet_data["MetExcExpPerc"], errors='coerce')
                if sheet_name in ['MathPerformanceTrends', 'ELAPerformanceTrends']:
                    sheet_data = sheet_data[sheet_data['Grades'].str.contains("Grade", na=False)]
                    sheet_data['StartYear'] = sheet_data['SchoolYear'].str.split('-').str[1].astype(int)
                    sheet_data['Grade'] = sheet_data['Grades'].str.extract(r'(\d+)').astype(int)
                    sheet_data['GraduationYear'] = sheet_data['StartYear'] + (12 - sheet_data['Grade'])
                    sheet_data.drop(columns=['StartYear', 'Grade', 'Grades'], inplace=True)
                    if sheet_name == 'MathPerformanceTrends':
                        sheet_data["Subject"] = "Math"
                    if sheet_name == 'ELAPerformanceTrends':
                        sheet_data["Subject"] = "ELA"
                if "DistrictPercent" in sheet_data.columns:
                    sheet_data["DistrictPercent"] = pd.to_numeric(sheet_data["DistrictPercent"], errors='coerce')

                if 'DistrictCode' in sheet_data.columns:
                    sheet_data.rename(columns={'DistrictCode': 'DIST'}, inplace=True)
                if 'DIST' in sheet_data.columns:
                    sheet_data.set_index('DIST', inplace=True)
                    sheet_data.index = sheet_data.index.astype(str).str.strip().str.lstrip('0')
                sheet_data = sheet_data[columns_mapping[sheet_name]]     
                cleaned_data[sheet_name] = sheet_data
            
        
        return cleaned_data
    
    except ValueError as e:
        print(f"Error processing {filename}. {e}")
        return None
    except Exception as e:
        print(f"Error processing {filename}. {e}")
    
def filter_and_pivot(data, dfg=None, coname=None, group=None, subject=None, distname=None, base_name="report", alt=None, abs_data=None, school_performance_by_grade=None):
    filtered_data = None
    filtered_abs_data = None
    filtered_dist_scores_bygrade = None
    text_title = (
        f"DFG: {dfg} | DISTNAME: {distname} | "
        f"CONAME: {coname} | GROUP: {group} | "
        f"SUBJECT: {subject}"
    )    

    # Separate data for the specified DISTNAME without other filters
    distname_data = data[(data.DISTNAME == distname) & (data.Subject == subject)]
    distname_abs_data = abs_data[(abs_data.DISTNAME == distname)]
    dist_scores_bygrade = school_performance_by_grade[(school_performance_by_grade.DISTNAME == distname) & (school_performance_by_grade.Subject == subject)]
    # Filter the rest of the data, excluding the specified DISTNAME, only if data is not None
    filtered_data = data[data.DISTNAME != distname] if data is not None else pd.DataFrame()
    filtered_abs_data = abs_data[abs_data.DISTNAME != distname] if abs_data is not None else pd.DataFrame()
    filtered_dist_scores_bygrade = school_performance_by_grade[school_performance_by_grade.DISTNAME != distname] if school_performance_by_grade is not None else pd.DataFrame()

    # Apply additional filters if specified and if filtered data is valid
    filters = {'DFG': dfg, 'CONAME': coname, 'GROUP': group, 'Subject': subject}
    
    for key, value in filters.items():
        if value is not None:
            if not filtered_data.empty and key in filtered_data.columns:
                if key == 'DFG' and isinstance(value, list):
                    filtered_data = filtered_data[filtered_data.DFG.isin(value)]
                    filtered_dist_scores_bygrade = filtered_dist_scores_bygrade[filtered_dist_scores_bygrade.DFG.isin(value)]
                else:
                    filtered_data = filtered_data[filtered_data[key] == value]
                    filtered_dist_scores_bygrade = filtered_dist_scores_bygrade[filtered_dist_scores_bygrade[key] == value]
                    
            if not filtered_abs_data.empty and key in filtered_abs_data.columns:
                if key == 'DFG' and isinstance(value, list):
                    filtered_abs_data = filtered_abs_data[filtered_abs_data.DFG.isin(value)]
                else:
                    filtered_abs_data = filtered_abs_data[filtered_abs_data[key] == value]

    # Combine the unfiltered DISTNAME data with the filtered data
    data = pd.concat([distname_data, filtered_data]) if not distname_data.empty else filtered_data
    abs_data = pd.concat([distname_abs_data, filtered_abs_data]) if not distname_abs_data.empty else filtered_abs_data
    grade_data = pd.concat([dist_scores_bygrade, filtered_dist_scores_bygrade]) if not dist_scores_bygrade.empty else filtered_dist_scores_bygrade

    parts = [base_name]
    parts.append(f"{distname.replace(' ', '_')}")
    if dfg is not None:
        parts.append(f"DFG-{','.join(dfg) if isinstance(dfg, list) else dfg}")
    if coname is not None:
        parts.append(f"CONAME-{coname}")
    if group is not None:
        parts.append(f"GROUP-{group.replace('/', '_')}")
    if subject is not None:
        parts.append(f"SUBJECT-{subject}")
    
    # Join all parts with underscores to form the filename
    filename = "_".join(parts) + ".png"

    # Pivot the data
    pivot_data = data.pivot(index='DISTNAME', columns='SchoolYear', values='ProficiencyRateforFederalAccountability').fillna(0)
    desired_grade_order = ['PK', 'KG', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
    absentee_by_grade_pivot_data = abs_data.pivot_table(
        index=['DISTNAME', 'SchoolYear'],
        columns='Grade',
        values='DistrictPercent',
        aggfunc='mean',  # Aggregation function if needed
        fill_value=0
    )
    scores_by_grade_pivot_data = grade_data.pivot_table(
        index=['DISTNAME', 'SchoolYear'],
        columns='GraduationYear',
        values='MetExcExpPerc',
        aggfunc='mean',  # Aggregation function if needed
        fill_value=0
    )
    scores_by_grade_pivot_data = scores_by_grade_pivot_data.reindex(
        sorted(scores_by_grade_pivot_data.columns, reverse=True), 
        axis=1
    )
    # Filter the desired order to include only the grades present in the DataFrame
    existing_columns = [col for col in desired_grade_order if col in absentee_by_grade_pivot_data.columns]
    
    # Reindex to arrange columns in the specified order, only for existing columns
    absentee_by_grade_pivot_data = absentee_by_grade_pivot_data.reindex(columns=existing_columns)

    # Mean for the filtered data should only include the comparison district if it is included in that group
    if alt:
        mean_filtered = filtered_data.groupby('SchoolYear')['ProficiencyRateforFederalAccountability'].mean()
    else:
        mean_filtered = data.groupby('SchoolYear')['ProficiencyRateforFederalAccountability'].mean()
    return data, pivot_data, mean_filtered, re.sub(r'[\/\\:*?"<>|\s]', '_', filename.lower()), text_title, absentee_by_grade_pivot_data, scores_by_grade_pivot_data

def plot_proficiency_rate(districts, school_years, df_pivot, mean_data, mean_state, subject, last_year, filename, text_title):
    bar_width = 0.15
    x = np.arange(len(districts))  # X locations for districts

    fig, ax = plt.subplots(figsize=(12, 8))
    for i, year in enumerate(school_years):
        # Calculate bar positions for each year
        bars = ax.bar(x + i * bar_width, df_pivot[year].reindex(districts), width=bar_width, label=f"{subject} {year}")
        # Add text labels on each bar
        for bar in bars:
            yval = bar.get_height()
            ax.text(bar.get_x() + bar.get_width() / 2, yval, round(yval, 2), ha='center', va='bottom')
    text_title = f"{text_title}\n{subject} Proficiency Rate Year Over Year by District"
    # Add mean lines for the last year
    ax.axhline(y=mean_data[last_year], color='black', linestyle='--', label=f"Filtered Mean {last_year}")
    ax.axhline(y=mean_state[last_year], color='red', linestyle='--', label=f"State Mean {last_year}")

    # Formatting the plot
    
    ax.set_title(text_title)
    ax.set_xlabel('District')
    ax.set_ylabel('Proficiency Rate')
    ax.set_xticks(x + bar_width * (len(school_years) - 1) / 2)
    ax.set_xticklabels(districts, rotation=90)
    ax.legend()
    plt.tight_layout()
    plt.savefig(f"CHARTS/{filename.lower()}", format="png", dpi=300)
    plt.show()

def plot_indicator(pp_data, df3, indicator, title, y_label, bar_label, filename, text_title):
    mean_filtered = pp_data[indicator].mean()
    mean_state = df3[indicator].drop_duplicates().mean()
    
    x = np.arange(len(pp_data['DISTNAME']))  # The label locations
    bar_width = 0.25  # Width of each bar

    fig, ax = plt.subplots(figsize=(10, 6))

    # Create bars for the specified indicator
    ax.bar(x, pp_data[indicator], width=bar_width, label=bar_label)
    ax.axhline(y=mean_filtered, color='black', linestyle='--', label="Filtered Mean")
    ax.axhline(y=mean_state, color='red', linestyle='--', label="State Mean")
    text_title = f"{text_title}\n{title}"
    # Add labels, title, and legend
    ax.set_xlabel("DISTNAME")
    ax.set_ylabel(y_label)
    ax.set_title(text_title)
    ax.set_xticks(x)
    ax.set_xticklabels(pp_data['DISTNAME'], rotation=90)
    ax.legend(title="Indicators")
    plt.tight_layout()
    #print(f"Generating: {filename.lower()}")
    plt.savefig(f"CHARTS/{filename.lower()}", format="png", dpi=300)
    plt.show()

def plot_absentee_data(absentee_by_grade_pivot_data, title, filename, text_title):
    mask = absentee_by_grade_pivot_data == 0
    #plt.figure(figsize=(16, 10))
    text_title = f"{text_title}\n{title}"
    # Plot the heatmap
    rows, cols = absentee_by_grade_pivot_data.shape
    plt.figure(figsize=(16, rows/4))
    sns.heatmap(
        absentee_by_grade_pivot_data,
        annot=True,  # Annotate cells with the data values
        fmt=".1f",  # Format for annotating numbers
        cmap="RdYlGn_r",  # Color map
        linewidths=.5,  # Lines between cells
        cbar_kws={'label': 'District Percent'},
        vmax=20,
        mask=mask
    )
    
    # Set plot labels and title
    plt.xlabel("Grade")
    plt.ylabel("District - School Year")
    plt.title(text_title)
    plt.tight_layout()
    plt.savefig(f"CHARTS/{filename.lower()}", format="png", dpi=300, bbox_inches='tight')
    plt.show()   

def plot_grade_report(gpivot, title, filename, text_title, subject):
    mask = gpivot == 0
    rows, cols = gpivot.shape
    plt.figure(figsize=(14, rows/4))
    text_title = f"{text_title}\n{title}"
    # Plot the heatmap
    sns.heatmap(
        gpivot,
        annot=True,  # Annotate cells with the data values
        fmt=".1f",  # Format for annotating numbers
        #cmap="YlGnBu",  # Color map
        cmap="RdYlGn",
        linewidths=.5,  # Lines between cells
        cbar_kws={'label': 'District Percent'},
        mask=mask,
        annot_kws={"size": 8}
    )
    
    # Set plot labels and title
    plt.xlabel("Grade")
    plt.ylabel("District - Graduation Year")
    plt.title(text_title)
    plt.tight_layout()
    plt.savefig(f"CHARTS/{filename.lower()}", format="png", dpi=300, bbox_inches='tight')
    plt.show()

def load_tges_file(year):
    # Define URLs and file paths
    zip_url = f'https://www.nj.gov/education/guide/{year}/TGES{str(year)[-2:]}_Zipped.zip'
    local_zip_path = f'DATA/TGES{year}_Zipped.zip'
    local_csv_path = f'DATA/TGES{year}_SUMYR5.csv'
    
    # Check if the CSV file is cached locally
    if os.path.exists(local_csv_path):
        print(f"Loading {local_csv_path} from cache.")
        df = pd.read_csv(local_csv_path, encoding='cp1252').set_index('DIST')
        return df

    # If not cached, download and extract the file
    print(f"Downloading {zip_url}...")
    response = requests.get(zip_url)
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
        file_names = zip_file.namelist()
        
        # Use fnmatch to find any file that ends with 'SUMYR5.CSV' in any subdirectory
        target_file = next((name for name in file_names if fnmatch.fnmatch(name, '**/SUMYR5.CSV')), None)
        
        # Check if the target file was found
        if target_file:
            with zip_file.open(target_file) as extracted_file:
                # Load into DataFrame and cache locally
                df = pd.read_csv(extracted_file, encoding='cp1252').set_index('DIST')
                df.to_csv(local_csv_path)  # Save to local cache
                print(f"File cached as {local_csv_path}.")
        else:
            print(f"File ending with 'SUMYR5.CSV' not found in the ZIP archive.")
            df = None
    
    return df

def download_yearly_performance_report(year_range):
    # Construct the URL based on the input year range
    base_url = "https://navilp7rg08njprsharedst.blob.core.windows.net/perf-reports-ct/DataFiles/"
    file_name = "Database_DistrictStateDetail.xlsx"
    url = f"{base_url}{year_range}/{file_name}"
    
    # Define the local filename to save the downloaded file
    local_filename = f"DATA/{year_range.replace('-', '_')}_DistrictStateDetail.xlsx"
    
    # Check if the file is already downloaded
    if os.path.exists(local_filename):
        print(f"Loading {local_filename} from cache.")
        return local_filename

    # Download the file if not cached
    print(f"Downloading {url}...")
    response = requests.get(url)
    
    # Check if the download was successful
    if response.status_code == 200:
        # Write content to a local file
        with open(local_filename, 'wb') as file:
            file.write(response.content)
        print(f"File downloaded and saved as {local_filename}.")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")
        return None

    return local_filename

def get_adjacent_dfgs(df, current_dfg):
    dfg_values = df['DFG'].dropna().astype(str)
    sorted_dfgs = sorted(dfg_values.unique())
    try:
        index = sorted_dfgs.index(current_dfg)
    except ValueError:
        print(f"DFG '{current_dfg}' not found.")
        return None, None
    next_dfg = sorted_dfgs[index + 1] if index < len(sorted_dfgs) - 1 else None
    
    return next_dfg

def clean_and_deduplicate(df, subset, new_index):
    df = df.reset_index()  # Reset index to allow for deduplication
    df = df.drop_duplicates(subset=subset)
    return df.set_index(new_index)

# Helper Function: Clean and preprocess district details
def get_district_details(summary_data, dist_name):
    district_details = summary_data[summary_data.DISTNAME == dist_name]
    dist_county = str(district_details['CONAME'].iloc[0])
    dist_dfg = str(district_details['DFG'].iloc[0])
    dist_group = str(district_details['GROUP'].iloc[0])
    return district_details, dist_county, dist_dfg, dist_group

# Helper Function: Process and pivot data
def process_and_plot_data(config, df3, df_abs, df_by_grade):
    filtered_data, df_pivot, mean_filtered, filename, text_title, absentee_by_grade_pivot_data, by_grade_pivot = filter_and_pivot(
        df3,
        dfg=config['dfg'],
        coname=config['coname'],
        group=config['group'],
        subject=config['subject'],
        distname=config['distname'],
        abs_data=df_abs,
        school_performance_by_grade=df_by_grade
    )

    # State-wide mean proficiency rates
    mean_state = (
        df3[df3['Subject'] == config['subject']]
        .groupby('SchoolYear')['ProficiencyRateforFederalAccountability']
        .mean()
    )

    # Unique districts and school years
    districts = filtered_data['DISTNAME'].unique()
    school_years = sorted(filtered_data['SchoolYear'].unique())
    last_year = school_years[-1]

    # Plot proficiency rate chart
    plot_proficiency_rate(
        districts, school_years, df_pivot, mean_filtered, mean_state, config['subject'], last_year, filename, text_title
    )

    # Save pivoted data
    df_pivot.to_csv(f"CHARTS/{filename.lower()}.csv")

    # Plot by grade report
    plot_grade_report(
        gpivot=by_grade_pivot,
        title="Performance by Graduation Year",
        filename=f"{filename.replace('.png', '_bygradyear.png')}",
        text_title=text_title,
        subject=config['subject']
    )

    # Additional charts for ELA
    if config['subject'] == 'ELA':
        pp_data = filtered_data[['IND16A', 'IND16B', 'IND18A', 'IND18B', 'DISTNAME', 'ENROLL', 'PP']].drop_duplicates()

        # Plot various indicators
        indicators = [
            ('IND18A', "Pupils per Administrator", "Pupils per Admin", "_app.png"),
            ('IND16A', "Pupils per Staff", "Pupils per Staff", "_spp.png"),
            ('PP', "Per Pupil Cost", "Per Pupil", "_pp.png"),
            ('IND16B', "Staff Mean Salary", "Staff Mean Salary", "_ss.png")
        ]

        for indicator, title, bar_label, suffix in indicators:
            plot_indicator(
                pp_data, df3, indicator=indicator,
                title=title, y_label="Values", bar_label=bar_label,
                filename=f"{filename.replace('_subject-ela.png', suffix)}",
                text_title=text_title
            )

        pp_data.to_csv(f"CHARTS/{filename.lower().replace('_subject-ela.png', '.csv')}")

        # Plot absenteeism data
        plot_absentee_data(
            absentee_by_grade_pivot_data,
            title="Chronic Absentee % by Grade",
            filename=f"{filename.replace('_subject-ela.png', '_abs.png')}",
            text_title=text_title.replace(' | SUBJECT: ELA', '')
        )

def ensure_directories_exist():
    """
    Ensures the 'DATA' and 'CHARTS' subfolders exist in the current working directory.
    If they do not exist, they are created.
    """
    required_dirs = ["DATA", "CHARTS"]
    for directory in required_dirs:
        if not os.path.exists(directory):
            os.makedirs(directory)
            
ensure_directories_exist()

## Pull in Data Files & Generate Reports

In [ ]:
# Load summary data and preprocess it
summary_data = load_tges_file(year)
summary_data.index = summary_data.index.astype(str).str.strip().str.lstrip('0')
summary_data['PP'] = (summary_data["IND1"] / summary_data["ENROLL"]).astype(int)

# Define a mapping for sheet names to variable names
sheet_mapping = {
    "ELAMathPerformanceTrends": "school_performance",
    "MathPerformanceTrends": "school_performance_by_grade",
    "ELAPerformanceTrends": "school_performance_by_grade",
    "ChronicAbsByGrade": "absentee_by_grade",
    "ChronicAbsenteeismByGrade": "absentee_by_grade"
}

# Initialize DataFrames
dataframes = {key: pd.DataFrame() for key in sheet_mapping.values()}

# Process performance reports for each year range
for year_range in performance_years:
    year_perf = download_yearly_performance_report(year_range)
    sheet_data = load_and_clean_data(year_perf, year_range)
    
    # Combine data for each relevant sheet
    for sheet_name, variable_name in sheet_mapping.items():
        if sheet_name in sheet_data:
            dataframes[variable_name] = pd.concat([dataframes[variable_name], sheet_data[sheet_name]])

# Assign processed DataFrames to variables for further processing
school_performance = dataframes["school_performance"]
school_performance_by_grade = dataframes["school_performance_by_grade"]
absentee_by_grade = dataframes["absentee_by_grade"]

# Clean and deduplicate the school performance data
school_performance = clean_and_deduplicate(school_performance, subset=['DIST', 'SchoolYear', 'Subject'], new_index='DIST')

# Clean and deduplicate school performance by grade for ELA and Math
school_performance_by_grade = clean_and_deduplicate(
    school_performance_by_grade, subset=['DIST', 'SchoolYear', 'GraduationYear', 'Subject'], new_index='DIST'
)
# Clean and deduplicate absenteeism data
absentee_by_grade.index = absentee_by_grade.index.astype(str).str.strip().str.lstrip('0')
absentee_by_grade = clean_and_deduplicate(
    absentee_by_grade, subset=['DIST', 'SchoolYear', 'Grade'], new_index='DIST'
)

# Preprocess summary data and join additional datasets
district_details, dist_county, dist_dfg, dist_group = get_district_details(summary_data, dist_name)
df3 = summary_data.join(school_performance, how='left')
df_abs = summary_data.join(absentee_by_grade, how='left')
df_by_grade = summary_data.join(school_performance_by_grade, how='left')
df_by_grade['GraduationYear'] = df_by_grade['GraduationYear'].fillna(0).astype(int)

# Prepare global and alternate configurations
global_dfg = alt_dfg.copy()
next_dfg = get_adjacent_dfgs(summary_data, dist_dfg)
if next_dfg:
    global_dfg.append(next_dfg)

# Base configurations
base_configs = [
    {'dfg': None, 'distname': dist_name, 'coname': dist_county, 'group': dist_group, 'subject': subj, 'alt': False}
    for subj in ['Math', 'ELA']
]

dfg_configs = [
    {'dfg': dist_dfg, 'distname': dist_name, 'coname': None, 'group': dist_group, 'subject': subj, 'alt': False}
    for subj in ['Math', 'ELA']
]

county_configs = [
    {'dfg': None, 'distname': dist_name, 'coname': dist_county, 'group': None, 'subject': subj, 'alt': False}
    for subj in ['Math', 'ELA']
]

# Alternate configurations
alt_dfg_configs = [
    {'dfg': adf, 'distname': dist_name, 'coname': None, 'group': dist_group, 'subject': subj, 'alt': True}
    for adf in global_dfg
    for subj in ['Math', 'ELA']
]

alt_county_configs = [
    {'dfg': None, 'distname': dist_name, 'coname': acounty, 'group': dist_group, 'subject': subj, 'alt': True}
    for acounty in alt_county
    for subj in ['Math', 'ELA']
]

# Combine all configurations
filter_configs = base_configs + dfg_configs + county_configs + alt_dfg_configs + alt_county_configs

# Process each configuration
for config in filter_configs:
    process_and_plot_data(config, df3, df_abs, df_by_grade)
